# Setup

In [1]:
# S3 prefix
prefix = 'Scikit-wine'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [2]:
WORK_DIRECTORY = 'data'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY) )

In [3]:
train_input

's3://sagemaker-us-east-2-207384830943/Scikit-wine/data'

# Create SageMaker Scikit Estimator and Train Model

In [4]:
from sagemaker.sklearn.estimator import SKLearn
script_path = 'train_wine_gbt.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={})

In [5]:
sklearn.fit({'train': train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2019-03-25-23-44-09-824


2019-03-25 23:44:10 Starting - Starting the training job...
2019-03-25 23:44:11 Starting - Launching requested ML instances......
2019-03-25 23:45:12 Starting - Preparing the instances for training......
2019-03-25 23:46:26 Downloading - Downloading input data
2019-03-25 23:46:26 Training - Training image download completed. Training in progress.
2019-03-25 23:46:27,266 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2019-03-25 23:46:27,269 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-03-25 23:46:27,292 sagemaker_sklearn_container.training INFO     Invoking user training script.
2019-03-25 23:46:27,573 sagemaker-containers INFO     Module train_wine_gbt does not provide a setup.py. 
Generating setup.py
2019-03-25 23:46:27,573 sagemaker-containers INFO     Generating setup.cfg
2019-03-25 23:46:27,573 sagemaker-containers INFO     Generating MANIFEST.in
2019-03-25 23:46:27,574 sagemaker-containers INFO     Install

# Deploy it

In [12]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2019-03-25-23-44-09-824
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2019-03-25-23-44-09-824


--------------------------------------------------------------------------!

In [14]:
#get endpoint name
endpoint_name = predictor.endpoint
endpoint_name

'sagemaker-scikit-learn-2019-03-25-23-44-09-824'

# "Inference"/ Make Predictions

In [15]:
from sagemaker.predictor import json_serializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_JSON

## Summer Water

<img src="images/summer-water.png" width=75>

"Summer Water is more than the rosé of the season, it’s a state of mind - every drop full of pink-tinted possibility. This dry rosé reflects the pale pink promise of the perfect warm-weather wine. It’s perfectly crisp, lighthearted and destined for a good time. The grapes for this Central Coast Grenache and Syrah blend were picked early in the season to achieve a natural acidity that’s ideal for summer drinking. To ensure as little extraction from the skins as possible, we used the direct press method, retaining the delicate fruit aromas and pale color rosé is known for. The result is light, crisp, and dry - with notes of strawberry, pink grapefruit, rose, and orange blossom. Drink it chilled with the ones you love. Summer-friendly fare like shellfish, ceviche, and salads will pair swimmingly."

$18.00

In [16]:
feature_payload = {"description": "Summer Water is more than the rosé of the season, it’s a state of mind - every drop full of pink-tinted possibility. This dry rosé reflects the pale pink promise of the perfect warm-weather wine. It’s perfectly crisp, lighthearted and destined for a good time. The grapes for this Central Coast Grenache and Syrah blend were picked early in the season to achieve a natural acidity that’s ideal for summer drinking. To ensure as little extraction from the skins as possible, we used the direct press method, retaining the delicate fruit aromas and pale color rosé is known for. The result is light, crisp, and dry - with notes of strawberry, pink grapefruit, rose, and orange blossom. Drink it chilled with the ones you love. Summer-friendly fare like shellfish, ceviche, and salads will pair swimmingly.",
                   "price": 18.00}

predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=json_serializer,
    content_type=CONTENT_TYPE_JSON,
    accept=CONTENT_TYPE_JSON)

predictor.predict(data=feature_payload)

b'{"predicted_points": "87.0", "message": "Very good: a wine with special qualities."}'

## dom-perignon-legacy-2008

<img src="images/dom-perignon-legacy-2008.jpg" width=75>

There's power to this graceful Champagne, with the vivid acidity swathed in a fine, creamy mousse and flavors of toasted brioche, kumquat, pastry cream, candied ginger and poached plum that dance across the palate. An underpinning of smoky mineral gains momentum on the lasting finish. Drink now through 2033."

$180.00

In [18]:
feature_payload = {"description": "There's power to this graceful Champagne, with the vivid acidity swathed in a fine, creamy mousse and flavors of toasted brioche, kumquat, pastry cream, candied ginger and poached plum that dance across the palate. An underpinning of smoky mineral gains momentum on the lasting finish. Drink now through 2033.",
                   "price": 180.00}

predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=json_serializer,
    content_type=CONTENT_TYPE_JSON,
    accept=CONTENT_TYPE_JSON)

predictor.predict(data=feature_payload)

b'{"predicted_points": "92.0", "message": "Outstanding: a wine of superior character and style."}'

# Delete Endpoint

In [19]:
sklearn.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: sagemaker-scikit-learn-2019-03-25-23-44-09-824
